In [ ]:

# où: objet MoleculeInfo
class MoleculeInfo:
    .hamiltonian # MolecularHamiltonian
                 # MolecularHamiltonian(one_body, two_body, constant)
                 # méthode to_electronic_structure_hamiltonian()
    .noons
    .energies
    .n_el
    
# warning: for now, trotterize prend un Hamiltonien: sum_i lambda_i P_i,
# et suppose qu'on veut U approx exp(-i H)

def trotterize(observable, n_steps, order):
    """
    observable: H = sum_k theta_k T_k, theta_k : numérique ou symbolique
    
    Returns:
        approx de exp(-i H coeff)
    """
#transforms

# |0011> (fermionic): 2 en écriture décimale

# -> (JW) |0011> (2 en écriture décimale)
# -> (BK) ??
# -> (parity) ??

# état HF: |1100>
# si 3 electrons: |1110>
# si selection EA:
# si 3 electrons: |1110> (14)
# mais orbitale 0 et 3 sont inactives
# état HF "actif": |11> (2)
# enough in molecule_active? n_el: 2, n_orbs = 2, |11>.

In [ ]:
###########
# étape 0: chimie
###########
# contrat: we need h_h2, noons, energies, n_el
# si j'ai 2 electrons et 2 orbitales: 4 qubits
# orbitales ordonnées par énergie croissante
molecule = my_chemistry_computation(molecule_description)


###########
# étape A # (option)
###########
from qat.fermion.chemistry import select_active_space

# version fonctionnelle
# h_h2_active, active_indices = select_active_space(h_h2, noons, nels, criteria)
molecule_active = select_active_space(molecule, params)

# version "méthode"
molecule_active = molecule.select_active_space(params)
# ok parce que pas d'effet de bord, quitte à avoir flag inplace

molecule = molecule_active

###########
# étape B #
###########
# cluster_ops : list FermionHamiltonian Cc CCcc CCCccc : T_i
# fonctionnelle
cluster_ops = generate_cluster_ops(molecule.noons, molecule.energies, params)
# methode
cluster_ops = molecule.generate_cluster_ops(params)

theta0 = init_guess_mp2(molecule.hamiltonian, molecule.energies)

###########
# étape C #
###########
observable = molecule.hamiltonian.to_el_str_hamiltonian().to_fermion(method)
hf_int_f = recode_integer(molecule.get_hf(), method)
# cluster_ops: fermion to spin: see previous line

###########
# étape D #
###########
prep_circ = make_prep(hf_int_f)
circuit = trotterize(cluster_ops)  # U approx exp(-i sum_k theta_k T_k)
circuit_tot = prep_circ + circuit

# circuit var : U(theta) |psi0>

# VQE:
qpu = scipy_minimize_plugin(theta_0) | LinAlg()
res = qpu.submit(circuit.to_job(observable))

############
# black_box:
# ucc_preparation(molecule, params_active, encoding_method) -> circ, obs
############